## Function Approximation
### Using the PSO algorithm to optimise the ANN's parameters

**1. Import Resources**

In [1]:
# import resources
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

**2. Create Neural Network Class**

In [2]:
class NeuralNetwork:    
    def __init__(self, func, hiddenLayerNeurons, activation):
        self.func = func # name of the function to be optimized
        self.layerArch = list.copy(hiddenLayerNeurons) #list of neurons in the hidden layers
        self.activation = activation # name of the activation fuction
         
        # adding input and output layers to the hidden neurons list
        self.inputOutputNeurons()
        
        # calculate number of weights
        self.nWeights = self.numWeights()
        
        # set activation function
        self.actFunc = self.functionSelection()
        
        # set indices at which weight matrix to be split for matrix multiplication
        self.splitIndices = self.splitIndices()
        
    def getANN_Hyperparameters(self):
        return [self.func, self.activation, self.layerArch]
        
    def getnWeights(self): # getter methonds for number of weights
        return self.nWeights
        
    def inputOutputNeurons(self): # a private method to neuralNetwork class
        # Set number neurons in the Input layer
        if self.func in ('XOR','Complex'):
            self.layerArch.insert(0,2) # XOR & Complex functions are based on two input variables
        else:
            self.layerArch.insert(0,1) # Linear, Cubic, Sine, Tanh functions are based on single input variable
       
        # Set number of neurons in the Output layer
        self.layerArch.append(1) # only single output value is expected from the fuction
        
    def numWeights(self):# a private method to neuralNetwork class
        n_weights = sum(self.layerArch[i]*self.layerArch[i+1] for i in range(len(self.layerArch)-1))
        return n_weights
    
    def functionSelection(self):# a private method to neuralNetwork class
        return activation_funcs[self.activation]
    
    def splitIndices(self):
        indices = []
        for i in range(len(self.layerArch)-1):
            indices.append(self.layerArch[i]*self.layerArch[i+1])

        splitIndices = []
        splitIndices.append(indices[0])
        for j in range(1,len(indices)-1):
            splitIndices.append(splitIndices[j-1] + indices[j])

        return splitIndices
    
    def forward(self, weights,X,Y):
        
        inputArray = np.copy(X)
       
        wArray = np.split(weights,self.splitIndices)
        #print("splitIndices",self.splitIndices)
        #print("wArray",wArray)
        #print("layerArch", self.layerArch)
        
        wMatrix=[]
        for i in range(0,len(self.layerArch)-1):
            wMatrix.append(wArray[i].reshape(self.layerArch[i+1],self.layerArch[i]))
        #print("wArray after reshape", wMatrix)
    
        desiredArray = np.copy(Y)
        predictArray = []
        
        for i in range(len(inputArray)):
            ih = inputArray[i].reshape(-1,1)
            for j in range(0,len(self.layerArch)-1):
                #print("wMatrix = ", wMatrix[j], "ih = ", ih)
                ih = np.matmul(wMatrix[j],ih)
            
                if j != (len(self.layerArch)-1):
                    ih = self.actFunc(ih)
            predictArray.append(ih)
            
        #print(len(desiredArray), len(predictArray))

        squeezePredict = np.squeeze(predictArray)

        mse = ((desiredArray - squeezePredict)**2).mean(axis = None)
       
        return mse.tolist(), squeezePredict

**3. Helper function for passing activation function**

In [3]:
activation_funcs = {
    'Null': lambda x: x*0,
    'Sigmoid': lambda x: 1/(1 + np.exp(-x)),
    'Hyperbolic Tangent': lambda x: np.tanh(x),
    'Cosine': lambda x: np.cos(x),
    'Gaussian': lambda x: np.exp(-x**2/2),
        }

**4. Function to read data from the .txt file**

In [4]:
# load the training data from .txt file into a list
def read_data(func):
    txt_address = {'Linear':'Data/1in_linear_test.txt',
                   'Cubic':'Data/1in_cubic_test.txt',
                   'Sine':'Data/1in_sine_test.txt',
                   'TanH':'Data/1in_tanh_test.txt',
                   'XOR':'Data/2in_xor_test.txt',
                   'Complex':'Data/2in_complex_test.txt'}

    train_data_file = open(txt_address[func],'r')
    line_data = train_data_file.readlines()
    train_data_file.close()

    train_list = []
    for line in line_data:
        line = line.strip().split()
        for d in line:
            train_list.append(float(d))
        
    #check     
    #print(train_list)

    if func in ('XOR','Complex'):
        train_array = np.asarray(train_list, dtype=np.float32).reshape(-1,3)
        X,Y = train_array[:,0:2], train_array[:,-1]

    elif func in ('Linear', 'Cubic', 'Sine', 'TanH'):
        train_array = np.asarray(train_list, dtype=np.float32).reshape(-1,2)
        X,Y = train_array[:,0], train_array[:,-1]
    else:
        "error reading data"    
    #print(train_array)
    #print(X)
    #print(Y)
    
    return X,Y

**5. Select function to be Tested**

In [5]:
# Set function from User input
func = input('Choose the function name \n(Options: Linear, Cubic, Sine, TanH, XOR, Complex) = ')

Choose the function name 
(Options: Linear, Cubic, Sine, TanH, XOR, Complex) = Sine


**6. Run Test**

In [6]:
if func == 'Linear':
    hidden_layer_neurons = [10,8]
    activation = 'Hyperbolic Tangent' 
    weights = np.genfromtxt('linear_weights.csv',delimiter = ',')
elif func == 'Cubic':
    hidden_layer_neurons = [8,5]
    activation = 'Hyperbolic Tangent' 
    weights = np.genfromtxt('cubic_weights.csv',delimiter = ',')
elif func == 'Sine':
    hidden_layer_neurons = [8,5]
    activation = 'Hyperbolic Tangent' 
    weights = np.genfromtxt('sine_weights.csv',delimiter = ',')
elif func == 'TanH':
    hidden_layer_neurons = [10,8]
    activation = 'Hyperbolic Tangent' 
    weights = np.genfromtxt('tanh_weights.csv',delimiter = ',') 
elif func == 'XOR':
    hidden_layer_neurons = [8,5]
    activation = 'Hyperbolic Tangent' 
    weights = np.genfromtxt('xor_weights.csv',delimiter = ',') 
elif func == 'Complex':
    hidden_layer_neurons = [10,8]
    activation = 'Cosine' 
    weights = np.genfromtxt('complex_weights.csv',delimiter = ',') 
else: 'function not found'
    
X,Y = read_data(func) # read data
ann_object = NeuralNetwork(func, hidden_layer_neurons, activation) #create instance of neural network
mse, predict_array = ann_object.forward(weights,X,Y) #run ann


**7.Print Result**

In [7]:
#Print Result
print('Mean Squared Error: {:0.4f}'.format(mse))
accu = (1-mse)*100
print('\nAccuracy:{:0.2f}%'.format(accu))
print("\nPredicted Array\n",predict_array)
print("\nDesired output\n",Y)
#print("\nInput from test file\n",X)
print("\nTrained weights used:\n",weights)

Mean Squared Error: 1.3135

Accuracy:-31.35%

Predicted Array
 [-0.22347606  0.88730383  0.51171442 -0.79644697 -0.00794621  0.71079885
 -0.86298896 -0.82833213  0.75462312  0.66426423 -0.0466703  -0.12813016
 -0.84644642  0.82509985 -0.83935253 -0.67253308 -0.69827814  0.65157156
 -0.88491764  0.24688917 -0.88689002 -0.79522612  0.88491764  0.12056671
  0.8666133  -0.23442357 -0.87904723  0.88567166  0.88618275 -0.78065453]

Desired output
 [ 8.6700886e-02 -4.2888781e-01 -1.5039206e-01  6.3619179e-01
 -8.3633435e-01 -6.8664336e-01  3.3489913e-01  6.0654324e-01
 -6.6432232e-01 -7.0558351e-01  9.9999997e-05  8.1760991e-01
  5.8377165e-01 -6.1004549e-01  5.9341800e-01  1.9640167e-01
  2.0612091e-01 -7.1014535e-01 -1.0000000e+00 -7.9985160e-01
  4.6617243e-01  2.5743604e-01  1.0000000e+00 -8.1868756e-01
 -5.4797947e-01  8.9250840e-02  5.1261735e-01 -4.7820482e-01
 -4.1524184e-01  6.4787400e-01]

Trained weights used:
 [-0.38378805 -0.05842995  0.03735707  2.50929737  0.6298905  -0.2124604